In [1]:
import numpy as np
import os
import librosa
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential 
from keras.layers import  Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
file_path="audio"
max_pad_length=40
random_state=42
labels=['stop', 'left', 'unknown', 'off', 'yes', 'up', 'on', 'down', 'no', 'go', 'right']

C:\Users\abdal_000\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def test_wavfile(path="audio/go/0d53e045_nohash_1.wav"):
    
    sample_rate, samples = wavfile.read(path)
    frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)
    plt.pcolormesh(times, frequencies, spectrogram)
    plt.imshow(spectrogram)
    plt.ylabel('Frequecy [HZ]')
    plt.xlabel('Time [sec]')
    plt.show()

In [3]:
def convert_wave_to_mfcc(filePath):
    wav, sr =librosa.load(filePath, mono = True, sr=None)
    mfcc = librosa.feature.mfcc(wav, sr=sr)
    ##padding
    mfcc_feature = np.pad(mfcc, pad_width=((0, 0), (0, max_pad_length - mfcc.shape[1])), mode='constant')
    return mfcc_feature

In [4]:
def get_labels(path=file_path):
    return os.listdir(path)

In [5]:
def save_mfcc_files(path=file_path):
    labels = get_labels(path)
    #print(labels)
    for label in labels:
        mfcc_features=[]
        audfiles=[path+'/'+label+'/'+aud for aud in os.listdir(path+'/'+label)]
        for audio in audfiles:
            mfcc_features.append(convert_wave_to_mfcc(audio))
        #print(len(mfcc_vectors))
        np.save(label+'.npy', mfcc_features)

In [6]:
get_labels()

['stop',
 'left',
 'unknown',
 'off',
 'yes',
 'up',
 'on',
 'down',
 'no',
 'go',
 'right']

In [6]:
def generate_training_testing_set(path=file_path,test_size=.3):
    labels = get_labels(path)
    #features=[]
    features=np.load(labels[0]+'.npy')
    classes=np.zeros(features.shape[0])
    #for i,label in zip(range(0,len(labels)-1),labels):
    for i in range(1,len(labels)):
        x=np.load(labels[i]+'.npy')
        features=np.vstack((features,x))
        classes=np.append(classes, np.full(x.shape[0], fill_value= (i)))
    return train_test_split(features, classes, test_size= test_size, random_state=random_state, shuffle=True)

In [6]:
#X_train, X_test, y_train, y_test = generate_training_testing_set()
#print(X_train.shape)
model =Sequential()
model.add(Conv2D(32,kernel_size=(2,2),activation='relu',input_shape=(20,40,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.2))
model.add(Conv2D(64,kernel_size=(2,2),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.2))
model.add(Conv2D(128,kernel_size=(2,2),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(.2))
model.add(Dense(11,activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.adadelta(),
              metrics=['accuracy'])


In [73]:
print ( len(X_train), len(X_test))

17003 7287


In [74]:
checkpointer = ModelCheckpoint(filepath='best_weights.hdf5',verbose=1,save_best_only=True)
model.fit(X_train.reshape(X_train.shape[0],20,40,1), to_categorical(y_train),
          batch_size=256, epochs=250, verbose=1,validation_split=0.3,callbacks=[checkpointer] )

Train on 11902 samples, validate on 5101 samples
Epoch 1/250
11902/11902 [==============================] - 2s 156us/step - loss: 6.4468 - acc: 0.0974 - val_loss: 2.3939 - val_acc: 0.0992
Epoch 2/250
11902/11902 [==============================] - 1s 101us/step - loss: 2.4484 - acc: 0.1028 - val_loss: 2.3897 - val_acc: 0.0972
Epoch 3/250
11902/11902 [==============================] - 1s 102us/step - loss: 2.4026 - acc: 0.0981 - val_loss: 2.3859 - val_acc: 0.0931
Epoch 4/250
11902/11902 [==============================] - 1s 102us/step - loss: 2.3947 - acc: 0.0987 - val_loss: 2.3827 - val_acc: 0.0927
Epoch 5/250
11902/11902 [==============================] - 1s 101us/step - loss: 2.3837 - acc: 0.1003 - val_loss: 2.3800 - val_acc: 0.0931
Epoch 6/250
11902/11902 [==============================] - 1s 101us/step - loss: 2.3813 - acc: 0.0971 - val_loss: 2.3779 - val_acc: 0.0929
Epoch 7/250
11902/11902 [==============================] - 1s 101us/step - loss: 2.3784 - acc: 0.0982 - val_loss: 2.3

Epoch 27/250
11902/11902 [==============================] - 1s 102us/step - loss: 1.9805 - acc: 0.2647 - val_loss: 1.7891 - val_acc: 0.3448
Epoch 28/250
11902/11902 [==============================] - 1s 101us/step - loss: 1.8581 - acc: 0.3268 - val_loss: 1.6288 - val_acc: 0.4105
Epoch 29/250
11902/11902 [==============================] - 1s 101us/step - loss: 1.7590 - acc: 0.3683 - val_loss: 1.4268 - val_acc: 0.5324
Epoch 30/250
11902/11902 [==============================] - 1s 99us/step - loss: 1.6431 - acc: 0.4105 - val_loss: 1.4401 - val_acc: 0.5075
Epoch 31/250
11902/11902 [==============================] - 1s 101us/step - loss: 1.5356 - acc: 0.4585 - val_loss: 1.2330 - val_acc: 0.6179
Epoch 32/250
11902/11902 [==============================] - 1s 101us/step - loss: 1.4304 - acc: 0.4938 - val_loss: 1.1994 - val_acc: 0.5766
Epoch 33/250
11902/11902 [==============================] - 1s 101us/step - loss: 1.3541 - acc: 0.5257 - val_loss: 1.0387 - val_acc: 0.6679
Epoch 34/250
11902/11

11902/11902 [==============================] - 1s 101us/step - loss: 0.7398 - acc: 0.7447 - val_loss: 0.5136 - val_acc: 0.8377
Epoch 54/250
11902/11902 [==============================] - 1s 101us/step - loss: 0.7247 - acc: 0.7531 - val_loss: 0.5033 - val_acc: 0.8432
Epoch 55/250
11902/11902 [==============================] - 1s 101us/step - loss: 0.7057 - acc: 0.7580 - val_loss: 0.4911 - val_acc: 0.8459
Epoch 56/250
11902/11902 [==============================] - 1s 102us/step - loss: 0.7006 - acc: 0.7598 - val_loss: 0.4830 - val_acc: 0.8467
Epoch 57/250
11902/11902 [==============================] - 1s 101us/step - loss: 0.6877 - acc: 0.7679 - val_loss: 0.4753 - val_acc: 0.8479
Epoch 58/250
11902/11902 [==============================] - 1s 101us/step - loss: 0.6913 - acc: 0.7663 - val_loss: 0.4658 - val_acc: 0.8532
Epoch 59/250
11902/11902 [==============================] - 1s 101us/step - loss: 0.6710 - acc: 0.7739 - val_loss: 0.4438 - val_acc: 0.8590
Epoch 60/250
11902/11902 [=======

Epoch 80/250
11902/11902 [==============================] - 1s 102us/step - loss: 0.5110 - acc: 0.8293 - val_loss: 0.3521 - val_acc: 0.8857
Epoch 81/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.5119 - acc: 0.8251 - val_loss: 0.3602 - val_acc: 0.8810
Epoch 82/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.5032 - acc: 0.8324 - val_loss: 0.3555 - val_acc: 0.8832
Epoch 83/250
11902/11902 [==============================] - 1s 101us/step - loss: 0.4935 - acc: 0.8292 - val_loss: 0.3413 - val_acc: 0.8894
Epoch 84/250
11902/11902 [==============================] - 1s 101us/step - loss: 0.4981 - acc: 0.8305 - val_loss: 0.3403 - val_acc: 0.8912
Epoch 85/250
11902/11902 [==============================] - 1s 100us/step - loss: 0.4776 - acc: 0.8373 - val_loss: 0.3412 - val_acc: 0.8896
Epoch 86/250
11902/11902 [==============================] - 1s 100us/step - loss: 0.4844 - acc: 0.8327 - val_loss: 0.3450 - val_acc: 0.8890
Epoch 87/250
11902/119

Epoch 138/250
11902/11902 [==============================] - 1s 101us/step - loss: 0.3226 - acc: 0.8896 - val_loss: 0.2624 - val_acc: 0.9183
Epoch 139/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.3363 - acc: 0.8854 - val_loss: 0.2638 - val_acc: 0.9157
Epoch 140/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.3198 - acc: 0.8888 - val_loss: 0.2713 - val_acc: 0.9134
Epoch 141/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.3242 - acc: 0.8894 - val_loss: 0.2674 - val_acc: 0.9145
Epoch 142/250
11902/11902 [==============================] - 1s 101us/step - loss: 0.3139 - acc: 0.8949 - val_loss: 0.2590 - val_acc: 0.9167
Epoch 143/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.3094 - acc: 0.8957 - val_loss: 0.2790 - val_acc: 0.9110
Epoch 144/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.3168 - acc: 0.8909 - val_loss: 0.2780 - val_acc: 0.9092
Epoch 145/250
1190

11902/11902 [==============================] - 1s 99us/step - loss: 0.2399 - acc: 0.9171 - val_loss: 0.2486 - val_acc: 0.9200
Epoch 199/250
11902/11902 [==============================] - 1s 98us/step - loss: 0.2512 - acc: 0.9164 - val_loss: 0.2490 - val_acc: 0.9204
Epoch 200/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.2469 - acc: 0.9169 - val_loss: 0.2557 - val_acc: 0.9175
Epoch 201/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.2368 - acc: 0.9162 - val_loss: 0.2468 - val_acc: 0.9198
Epoch 202/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.2410 - acc: 0.9168 - val_loss: 0.2450 - val_acc: 0.9216
Epoch 203/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.2317 - acc: 0.9193 - val_loss: 0.2437 - val_acc: 0.9222
Epoch 204/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.2426 - acc: 0.9156 - val_loss: 0.2449 - val_acc: 0.9218
Epoch 205/250
11902/11902 [=======

Epoch 229/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.2206 - acc: 0.9257 - val_loss: 0.2420 - val_acc: 0.9233
Epoch 230/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.2175 - acc: 0.9264 - val_loss: 0.2397 - val_acc: 0.9226
Epoch 231/250
11902/11902 [==============================] - 1s 98us/step - loss: 0.2282 - acc: 0.9209 - val_loss: 0.2487 - val_acc: 0.9208
Epoch 232/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.2306 - acc: 0.9203 - val_loss: 0.2458 - val_acc: 0.9220
Epoch 233/250
11902/11902 [==============================] - 1s 98us/step - loss: 0.2151 - acc: 0.9270 - val_loss: 0.2439 - val_acc: 0.9239
Epoch 234/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.2058 - acc: 0.9315 - val_loss: 0.2382 - val_acc: 0.9230
Epoch 235/250
11902/11902 [==============================] - 1s 99us/step - loss: 0.2080 - acc: 0.9305 - val_loss: 0.2414 - val_acc: 0.9214
Epoch 236/250
11902/

In [7]:
model.load_weights('best_weights.hdf5')

In [76]:
y_predictions = [np.argmax(model.predict(np.expand_dims(example,axis=0))) for example in X_test.reshape(X_test.shape[0],20,40,1)]

accuracy =100*np.sum(np.array(y_predictions)==np.argmax( to_categorical(y_test),axis=1))/len(y_predictions)

print ('test accurac : %.4f%%' % accuracy)

test accurac : 92.0681%


In [8]:
def test (path):
    #path=r"C:\Users\abdal_000\Downloads\data-set\test\audio\clip_0a77ea19a.wav"
    mfcc_feature=convert_wave_to_mfcc(path)
    #print(mfcc_feature.shape)
    
    pre=model.predict(mfcc_feature.reshape(1,20,40,1))
    return np.argmax(pre)

In [9]:
import csv
audfiles=os.listdir(r"C:\Users\abdal_000\Downloads\data-set\test\audio")
print (audfiles[0])
with open('sample_submission.csv', 'w', newline='') as csvfile:
    fieldnames=['fname','label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    for audio in audfiles:
        pred = test(r"C:\Users\abdal_000\Downloads\data-set\test\audio\\"+audio)
        writer.writerow({'fname':audio,'label':labels[pred]})
    

clip_000044442.wav
